In [45]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [46]:
from gensim import corpora
documents = ["Human machine interface for lab abc computer applications",
             "A survey of user opinion of computer system response time",
             "The EPS user interface management system",
             "System and human system engineering testing of EPS",
             "Relation of user perceived response time to error measurement",
             "The generation of random binary unordered trees",
             "The intersection graph of paths in trees",
             "Graph minors IV Widths of trees and well quasi ordering",
             "Graph minors A survey"]
# documents = ["its easy to copy and do well"]

In [47]:
# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in documents]

# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

# texts = [[token for token in text if frequency[token] > 1] for text in texts]

from pprint import pprint  # pretty-printer
pprint(texts)

[['human', 'machine', 'interface', 'lab', 'abc', 'computer', 'applications'],
 ['survey', 'user', 'opinion', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'management', 'system'],
 ['system', 'human', 'system', 'engineering', 'testing', 'eps'],
 ['relation', 'user', 'perceived', 'response', 'time', 'error', 'measurement'],
 ['generation', 'random', 'binary', 'unordered', 'trees'],
 ['intersection', 'graph', 'paths', 'trees'],
 ['graph', 'minors', 'iv', 'widths', 'trees', 'well', 'quasi', 'ordering'],
 ['graph', 'minors', 'survey']]


In [48]:
dictionary = corpora.Dictionary(texts)
dictionary.save('./sims.dict')  # store the dictionary, for future reference
print(dictionary)

2018-03-25 19:36:24,392 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-03-25 19:36:24,394 : INFO : built Dictionary(35 unique tokens: ['abc', 'applications', 'computer', 'human', 'interface']...) from 9 documents (total 52 corpus positions)
2018-03-25 19:36:24,395 : INFO : saving Dictionary object under ./sims.dict, separately None
2018-03-25 19:36:24,397 : INFO : saved ./sims.dict


Dictionary(35 unique tokens: ['abc', 'applications', 'computer', 'human', 'interface']...)


In [49]:
print(dictionary.token2id)

{'abc': 0, 'applications': 1, 'computer': 2, 'human': 3, 'interface': 4, 'lab': 5, 'machine': 6, 'opinion': 7, 'response': 8, 'survey': 9, 'system': 10, 'time': 11, 'user': 12, 'eps': 13, 'management': 14, 'engineering': 15, 'testing': 16, 'error': 17, 'measurement': 18, 'perceived': 19, 'relation': 20, 'binary': 21, 'generation': 22, 'random': 23, 'trees': 24, 'unordered': 25, 'graph': 26, 'intersection': 27, 'paths': 28, 'iv': 29, 'minors': 30, 'ordering': 31, 'quasi': 32, 'well': 33, 'widths': 34}


In [50]:
new_doc = "Human computer interaction"
new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec)  # the word "interaction" does not appear in the dictionary and is ignored

[(2, 1), (3, 1)]


In [51]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('./sims.mm', corpus)  # store to disk, for later use
for c in corpus:
    print(c)

2018-03-25 19:36:24,763 : INFO : storing corpus in Matrix Market format to ./sims.mm
2018-03-25 19:36:24,765 : INFO : saving sparse matrix to ./sims.mm
2018-03-25 19:36:24,767 : INFO : PROGRESS: saving document #0
2018-03-25 19:36:24,768 : INFO : saved 9x35 matrix, density=16.190% (51/315)
2018-03-25 19:36:24,769 : INFO : saving MmCorpus index to ./sims.mm.index


[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)]
[(2, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)]
[(4, 1), (10, 1), (12, 1), (13, 1), (14, 1)]
[(3, 1), (10, 2), (13, 1), (15, 1), (16, 1)]
[(8, 1), (11, 1), (12, 1), (17, 1), (18, 1), (19, 1), (20, 1)]
[(21, 1), (22, 1), (23, 1), (24, 1), (25, 1)]
[(24, 1), (26, 1), (27, 1), (28, 1)]
[(24, 1), (26, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1)]
[(9, 1), (26, 1), (30, 1)]


In [52]:
from gensim import corpora, models, similarities

dictionary = corpora.Dictionary.load('./sims.dict')
corpus = corpora.MmCorpus('./sims.mm') # comes from the first tutorial, "Corpora and Vector Space"
print(corpus)

2018-03-25 19:36:24,887 : INFO : loading Dictionary object from ./sims.dict
2018-03-25 19:36:24,888 : INFO : loaded ./sims.dict
2018-03-25 19:36:24,890 : INFO : loaded corpus index from ./sims.mm.index
2018-03-25 19:36:24,891 : INFO : initializing cython corpus reader from ./sims.mm
2018-03-25 19:36:24,892 : INFO : accepted corpus with 9 documents, 35 features, 51 non-zero entries


MmCorpus(9 documents, 35 features, 51 non-zero entries)


In [53]:
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=2)

2018-03-25 19:36:25,017 : INFO : using serial LSI version on this node
2018-03-25 19:36:25,018 : INFO : updating model with new documents
2018-03-25 19:36:25,020 : INFO : preparing a new chunk of documents
2018-03-25 19:36:25,022 : INFO : using 100 extra samples and 2 power iterations
2018-03-25 19:36:25,023 : INFO : 1st phase: constructing (35, 102) action matrix
2018-03-25 19:36:25,024 : INFO : orthonormalizing (35, 102) action matrix
2018-03-25 19:36:25,034 : INFO : 2nd phase: running dense svd on (35, 9) matrix
2018-03-25 19:36:25,037 : INFO : computing the final decomposition
2018-03-25 19:36:25,038 : INFO : keeping 2 factors (discarding 57.737% of energy spectrum)
2018-03-25 19:36:25,039 : INFO : processed documents up to #9
2018-03-25 19:36:25,041 : INFO : topic #0(3.594): 0.579*"system" + 0.376*"user" + 0.270*"eps" + 0.257*"time" + 0.257*"response" + 0.230*"computer" + 0.224*"human" + 0.191*"interface" + 0.176*"survey" + 0.157*"opinion"
2018-03-25 19:36:25,043 : INFO : topic #1

In [54]:
doc = "Human computer interaction"
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow] # convert the query to LSI space
print(vec_lsi)

[(0, 0.453212762871565), (1, 0.03843944771149868)]


In [55]:
index = similarities.MatrixSimilarity(lsi[corpus]) # transform corpus to LSI space and index it

2018-03-25 19:36:25,756 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)
2018-03-25 19:36:25,759 : INFO : creating matrix with 9 documents and 2 features


In [56]:
index.save('./sims.index')

2018-03-25 19:36:25,869 : INFO : saving MatrixSimilarity object under ./sims.index, separately None
2018-03-25 19:36:25,871 : INFO : saved ./sims.index


In [57]:
sims = index[vec_lsi] # perform a similarity query against the corpus
print(list(enumerate(sims))) # print (document_number, document_similarity) 2-tuples

[(0, 0.99962217), (1, 0.9941941), (2, 0.9999954), (3, 0.9996517), (4, 0.9962734), (5, -0.055301595), (6, -0.036689866), (7, -0.03771901), (8, 0.16008198)]
